<h1 align=center><font size = 5>Capstone project on Segmenting and Clustering Neighborhoods in Toronto_Part2</font></h1>

## Problem 2:

I have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [78]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import requests
webUrl = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

print('Libraries imported.')

Libraries imported.


In [79]:
soup = BeautifulSoup(webUrl, 'xml')

In [80]:
table=soup.find('table')
#print(table)

In [81]:
#dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
data = pd.DataFrame(columns = column_names)

In [82]:
#Search all the postcode, borough, neighborhood 
for tr in table.find_all('tr'):
    rowData=[]
    for td in tr.find_all('td'):
        rowData.append(td.text.strip())
    if len(rowData)==3:
        data.loc[len(data)] = rowData
data.head(20)

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [83]:
data=data[data['Borough']!='Not assigned']

In [84]:
data[data['Neighborhood']=='Not assigned']=data['Borough']
data.head()

ValueError: cannot set using a list-like indexer with a different length than the value

In [85]:
temp_data=data.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_data=temp_data.reset_index(drop=False)
temp_data.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [86]:
data_merge = pd.merge(data, temp_data, on='Postalcode')

In [87]:
data_merge.drop(['Neighborhood'],axis=1,inplace=True)

In [88]:
data_merge.drop_duplicates(inplace=True)

In [89]:
data_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [90]:
data_merge.shape

(103, 3)

# fom the options using the link rather than geocode

In [91]:
geo_data=pd.read_csv('http://cocl.us/Geospatial_data')

In [92]:
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [93]:
geo_data.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_data, data_merge, on='Postalcode')
geo_merged.head(20)

,Postalcode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae
5,M1J,43.744734,-79.239476,Scarborough,Scarborough Village
6,M1K,43.727929,-79.262029,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,43.711112,-79.284577,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,43.716316,-79.239476,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,43.692657,-79.264848,Scarborough,"Birch Cliff, Cliffside West"


In [95]:
geo_merged.shape

(103, 5)